In [6]:
import pandas as pd
import numpy as np

np.random.seed(42)

n = 5000  
group_a = pd.DataFrame({
    'user_id': range(1, n+1),
    'group': 'A',
    'converted': np.random.binomial(1, 0.10, n)  
})

group_b = pd.DataFrame({
    'user_id': range(n+1, 2*n+1),
    'group': 'B',
    'converted': np.random.binomial(1, 0.12, n)  
})

ab_data = pd.concat([group_a, group_b], ignore_index=True)

ab_data.to_csv('ab_test_data.csv', index=False)
print("Dataset created with shape:", ab_data.shape)
ab_data.head()

Dataset created with shape: (10000, 3)


,user_id,group,converted
0,1,A,0
1,2,A,1
2,3,A,0
3,4,A,0
4,5,A,0


In [7]:
conversion_rates = ab_data.groupby('group')['converted'].agg(['count', 'sum', 'mean'])
conversion_rates['mean'] = conversion_rates['mean'].round(4) 
print(conversion_rates)

       count  sum    mean
group                    
A       5000  479  0.0958
B       5000  567  0.1134


In [8]:
from scipy import stats

contingency_table = pd.crosstab(ab_data['group'], ab_data['converted'])
print("Contingency Table:")
print(contingency_table)
print("\n")

chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)

print(f"Chi-Squared Statistic: {chi2:.4f}")
print(f"P-value: {p_value:.10f}") 

Contingency Table:
converted     0    1
group               
A          4521  479
B          4433  567


Chi-Squared Statistic: 8.0815
P-value: 0.0044720447


In [9]:
cr_a = conversion_rates.loc['A', 'mean']
cr_b = conversion_rates.loc['B', 'mean']

lift = (cr_b - cr_a) / cr_a

print(f"Conversion Rate (A): {cr_a:.4f}")
print(f"Conversion Rate (B): {cr_b:.4f}")
print(f"Absolute Difference: {cr_b - cr_a:.4f}")
print(f"Relative Lift: {lift:.2%}")

Conversion Rate (A): 0.0958
Conversion Rate (B): 0.1134
Absolute Difference: 0.0176
Relative Lift: 18.37%


In [12]:
print("--- A/B TEST RESULTS ---")
print(f"Control Conversion Rate (A): {cr_a:.2%}")
print(f"Treatment Conversion Rate (B): {cr_b:.2%}")
print(f"Relative Lift: {lift:.2%}")
print(f"P-value: {p_value:.4f}")
print("\n")
print("CONCLUSION & RECOMMENDATION:")
print("1.The increase in conversion rate for Variant B is statistically significant (p < 0.05).")
print("2.We recommend rolling out the new design to 100% of users.")
print(f"3.This change is projected to increase conversions by {lift:.2%}.")

--- A/B TEST RESULTS ---
Control Conversion Rate (A): 9.58%
Treatment Conversion Rate (B): 11.34%
Relative Lift: 18.37%
P-value: 0.0045


CONCLUSION & RECOMMENDATION:
1.The increase in conversion rate for Variant B is statistically significant (p < 0.05).
2.We recommend rolling out the new design to 100% of users.
3.This change is projected to increase conversions by 18.37%.
